<a href="https://colab.research.google.com/github/patsoong/CS506FinalProject/blob/main/notebooks/Stack_Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC
import xgboost as xgb
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score
import pandas as pd

features_df = pd.read_csv("team_season_features_v2_clean-2.csv")

num_cols = features_df.select_dtypes(include="number").columns.tolist() #remove features not to be used in training data
for col in ["champion", "season"]:
    if col in num_cols:
        num_cols.remove(col)

# add season-relative features (rank and z-score within each season)
for col in num_cols:
    features_df[f'{col}_season_rank'] = features_df.groupby('season')[col].rank(pct=True)
    features_df[f'{col}_season_zscore'] = features_df.groupby('season')[col].transform(
        lambda x: (x - x.mean()) / x.std() if x.std() > 0 else 0
    )

# update feature list
num_cols_extended = features_df.select_dtypes(include="number").columns.tolist()
for col in ["champion", "season"]:
    if col in num_cols_extended:
        num_cols_extended.remove(col)

X = features_df[num_cols_extended].copy()
X = X.replace([np.inf, -np.inf], np.nan)
y = features_df["champion"].astype(int)

#temporal split
train_mask = features_df["season"] <= 2015
X_train, X_test = X[train_mask], X[~train_mask]
y_train, y_test = y[train_mask], y[~train_mask]

id_test = features_df.loc[~train_mask, ["season", "team", "champion"]].copy()

log_reg = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("poly", PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)),
    ("scale", StandardScaler(with_mean=True, with_std=True)),
    ("clf", LogisticRegression(
        multi_class='ovr',
        class_weight='balanced',
        max_iter=10000,
        C=1,
        random_state=42
        ))
])

# svm_base = SVC(
#     kernel="rbf",
#     C=1,
#     gamma=0.01,
#     class_weight="balanced",
#     probability=False,
#     random_state=42
# )

# svm_cal = CalibratedClassifierCV(
#     svm_base,
#     method="sigmoid",
#     cv=3
# )

# svm = Pipeline([
#     ("impute", SimpleImputer(strategy="median")),
#     ("scale", StandardScaler(with_mean=True, with_std=True)),
#     ("clf", svm_cal),
# ])

xgboost = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("scale", StandardScaler(with_mean=True, with_std=True)),
    ("clf", xgb.XGBClassifier(
        objective='binary:logistic',
        eval_metric='auc',
        use_label_encoder=False,
        subsample=0.9,
        scale_pos_weight=25.548387096774192,
        n_estimators=100,
        min_child_weight=3,
        max_depth=5,
        learning_rate=0.01,
        gamma=0,
        colsample_bytree=0.8,
        random_state=42
    ))
])

estimators = [
    ("logreg", log_reg),
    # ("svm", svm),
    ("xgboost", xgboost)
]

# log_reg.fit(X_train, y_train)
# log_reg_probs = log_reg.predict_proba(X_test)[:, 1]
# svm.fit(X_train, y_train)
# svm_probs = svm.predict_proba(X_test)[:, 1]

# alpha = 1
# blend_probs = alpha * log_reg_probs + (1 - alpha) * svm_probs

# print("Blend ROC-AUC:", roc_auc_score(y_test, blend_probs))
# print("Blend Average Precision (PR-AUC):",
#       average_precision_score(y_test, blend_probs))

# blend_pred_labels = (blend_probs > 0.5).astype(int)
# print("Blend Binary Accuracy:", accuracy_score(y_test, blend_pred_labels))

# id_test["blend_proba_win"] = blend_probs

# idx = id_test.groupby("season")["blend_proba_win"].idxmax()

# blend_predicted_champs = (
#     id_test.loc[idx, ["season", "team", "blend_proba_win"]]
#           .rename(columns={"team": "team_pred",
#                            "blend_proba_win": "pred_prob"})
#           .reset_index(drop=True)
# )

# true_champs = (
#     id_test.loc[id_test["champion"] == 1, ["season", "team"]]
#           .rename(columns={"team": "team_true"})
#           .reset_index(drop=True)
# )

# blend_eval = blend_predicted_champs.merge(true_champs, on="season", how="left")
# blend_eval["correct"] = (blend_eval["team_pred"] == blend_eval["team_true"]).astype(int)

# print("Blend Top-1 accuracy:", blend_eval["correct"].mean())
# print("\nPredicted vs. True Champions by Season (Blend):")
# print(blend_eval.sort_values("season"))

stack_model = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(
        class_weight="balanced",
        max_iter=10000,
        random_state=42
    ),
    stack_method="predict_proba",
    cv=5,
    n_jobs=-1
)

stack_model.fit(X_train, y_train)

stack_probs = stack_model.predict_proba(X_test)[:, 1]

print("Stack ROC-AUC:", roc_auc_score(y_test, stack_probs))
print("Stack Average Precision (PR-AUC):", average_precision_score(y_test, stack_probs))

stack_pred_labels = (stack_probs > 0.5).astype(int)
print("Stack Binary Accuracy:", accuracy_score(y_test, stack_pred_labels))

id_test["stack_proba_win"] = stack_probs
id_test["stack_ranking_score"] = id_test["stack_proba_win"]

id_test["ranking_score"] = id_test["stack_proba_win"]

id_test["rank"] = (
    id_test.groupby("season")["ranking_score"]
           .rank(ascending=False, method="first")
)

# pick team with highest prob in each season
idx = id_test.groupby("season")["stack_proba_win"].idxmax()

stack_predicted_champs = (
    id_test.loc[idx, ["season", "team", "stack_proba_win"]]
          .rename(columns={"team": "team_pred", "stack_proba_win": "pred_prob"})
          .reset_index(drop=True)
)

true_champs = (
    id_test.loc[id_test["champion"] == 1, ["season", "team"]]
          .rename(columns={"team": "team_true"})
          .reset_index(drop=True)
)

stack_eval = stack_predicted_champs.merge(true_champs, on="season", how="left")
stack_eval["correct"] = (stack_eval["team_pred"] == stack_eval["team_true"]).astype(int)

print("Stack Top-1 accuracy:", stack_eval["correct"].mean())
print("\nPredicted vs. True Champions by Season (Stacked Model):")
print(stack_eval.sort_values("season"))

# ranks of the true champions in their seasons
true_champ_ranks = id_test.loc[id_test["champion"] == 1, "rank"]

# print top-k accuracies
k_values = [1, 2, 4]   # or whatever k’s you care about
print("\nTop-K Accuracy:")
for k in k_values:
    accuracy = (true_champ_ranks <= k).mean()
    print(f" Top-{k}: {accuracy:.4f}")

Stack ROC-AUC: 0.9806896551724138
Stack Average Precision (PR-AUC): 0.7740458572435316
Stack Binary Accuracy: 0.95
Stack Top-1 accuracy: 0.7

Predicted vs. True Champions by Season (Stacked Model):
   season team_pred  pred_prob  team_true  correct
0    2016  Warriors   0.982528  Cavaliers        0
1    2017  Warriors   0.983654   Warriors        1
2    2018  Warriors   0.981640   Warriors        1
3    2019   Raptors   0.983918    Raptors        1
4    2020     Bucks   0.500767     Lakers        0
5    2021  Clippers   0.733325      Bucks        0
6    2022  Warriors   0.980745   Warriors        1
7    2023   Nuggets   0.977562    Nuggets        1
8    2024   Celtics   0.983919    Celtics        1
9    2025   Thunder   0.983811    Thunder        1

Top-K Accuracy:
 Top-1: 0.7000
 Top-2: 0.9000
 Top-4: 1.0000
